<a href="https://colab.research.google.com/github/MichaelArgs/Dog-Breed-Classification/blob/main/dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.2 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import numpy as np
import streamlit as st
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [5]:
import zipfile

# Unzip the file
zip_path = '/content/70 dog breed.zip'  # Update this path if needed
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/70 dog breed')  # Extracts to a folder named "70 dog breed"


In [6]:
train_path = '/content/70 dog breed/train'
valid_path = '/content/70 dog breed/valid'
test_path = '/content/70 dog breed/test'

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [8]:
# Load MobileNetV2 with imagenet weights and include the input shape
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-50:]:  # Unfreeze the last 50 layers
    layer.trainable = True

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Use global average pooling instead of flattening
x = Dense(512, activation='relu')(x)  # Increased dense layer size
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with a low initial learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate if validation loss doesn't improve
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

callbacks = [early_stopping, reduce_lr]


In [11]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,  # Increased number of epochs
    callbacks=callbacks
)


Epoch 1/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 506s 2s/step - accuracy: 0.1363 - loss: 3.7759 - val_accuracy: 0.6100 - val_loss: 1.4664 - learning_rate: 1.0000e-04
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 554s 2s/step - accuracy: 0.5498 - loss: 1.6714 - val_accuracy: 0.7214 - val_loss: 1.0991 - learning_rate: 1.0000e-04
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 494s 2s/step - accuracy: 0.6659 - loss: 1.1771 - val_accuracy: 0.7271 - val_loss: 1.0471 - learning_rate: 1.0000e-04
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 499s 2s/step - accuracy: 0.7202 - loss: 0.9761 - val_accuracy: 0.7614 - val_loss: 0.9098 - learning_rate: 1.0000e-04
Epoch 5/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 495s 2s/step - accuracy: 0.7497 - loss: 0.8541 - val_accuracy: 0.7843 - val_loss: 0.8664 - learning_rate: 1.0000e-04
Epoch 6/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 491s 2s/step - accuracy: 0.7646 - loss: 0.7932 - val_accuracy: 0.8071 - val_loss: 0.7877 - learning_rate: 1.0000e-04
Epoch 7/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 489s 2s/step - acc

In [13]:
model.save("dog_breed_classifier.h5")

from google.colab import files
files.download("dog_breed_classifier.h5")

In [28]:
import json

# Get the breed names from train_generator
breed_names = train_generator.class_indices  # This is a dictionary mapping class names to indices

# Save breed names to a JSON file
with open("class_indices.json", "w") as f:
    json.dump(breed_names, f)

# Download the JSON file to your local machine
from google.colab import files
files.download("class_indices.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>